In [1]:
print("Hello Pydantic")


Hello Pydantic


[How to Make the Most of Pydantic](https://towardsdatascience.com/how-to-make-the-most-of-pydantic-aa374d5c12d)

In [32]:
from pydantic import BaseModel, ValidationError, Extra, Field
from functools import singledispatch
from typing import Optional


In [3]:
class Address(BaseModel):
    """
    Cat API Address definition
    """
    city: str
    zip_code: str
    number: int


class CatRequest(BaseModel):
    """
    Cat API Request definition
    """
    name: str
    age: int
    address: Address
 

In [4]:
my_json = {
    "name": "Lévy",
    "age": 3,
    "address": {
        "city": "Wonderland",
        "zip_code": "ABCDE",
        "number": 123
    }
}


In [5]:
my_json


{'name': 'Lévy',
 'age': 3,
 'address': {'city': 'Wonderland', 'zip_code': 'ABCDE', 'number': 123}}

In [6]:
data = CatRequest.parse_obj(my_json)


In [7]:
data


CatRequest(name='Lévy', age=3, address=Address(city='Wonderland', zip_code='ABCDE', number=123))

In [10]:
data.name  # Lévy


'Lévy'

In [9]:
data.address.number  # 123


123

In [12]:

bad_data = {
    "name": "Lévy",
    "age": "am I an age?",  # Note the type change here
    "address": {
        "city": "Wonderland",
        "zip_code": "ABCDE",
        "number": 123
    }
}


In [13]:
bad_data


{'name': 'Lévy',
 'age': 'am I an age?',
 'address': {'city': 'Wonderland', 'zip_code': 'ABCDE', 'number': 123}}

In [14]:
try:
    CatRequest.parse_obj(bad_data)
except ValidationError as err:
    print("Something went wrong with the data!")


Something went wrong with the data!


In [15]:
unnecessary_data = {
    "name": "Lévy",
    "age": 3,
    "key": "value",  # unnecessary
    "key2": "value2",  # unnecessary x2
    "address": {
        "city": "Wonderland",
        "zip_code": "ABCDE",
        "number": 123
    }
}


In [16]:
unnecessary_data


{'name': 'Lévy',
 'age': 3,
 'key': 'value',
 'key2': 'value2',
 'address': {'city': 'Wonderland', 'zip_code': 'ABCDE', 'number': 123}}

In [17]:

data = CatRequest.parse_obj(unnecessary_data)


In [18]:
data


CatRequest(name='Lévy', age=3, address=Address(city='Wonderland', zip_code='ABCDE', number=123))

In [20]:
class Address(BaseModel):
    """
    Cat API Address definition
    """
    class Config:
        extra = Extra.forbid

    # Note how we can even add descriptions to the fields!
    city: str = Field(..., description="Where the cat lives")
    zip_code: str
    number: int


class CatRequest(BaseModel):
    """
    Cat API Request definition
    """
    class Config:
        extra = Extra.forbid

    name: str
    age: int
    address: Address


In [21]:
data = CatRequest.parse_obj(unnecessary_data)


ValidationError: 2 validation errors for CatRequest
key
  extra fields not permitted (type=value_error.extra)
key2
  extra fields not permitted (type=value_error.extra)

In [23]:

@singledispatch
def process(model):
    """
    Default processing definition
    """
    raise NotImplementedError(f"I don't know how to process {type(model)}")


@process.register
def _(model: Address):
    """
    Handle addresses
    """
    print(f"Just got an address from {model.city}")


@process.register
def _(model: CatRequest):
    """
    Handle Cat Requests
    """
    print(f"Processing {model.name} the cat!")


In [26]:
address = Address(
    city="Wonderland",
    zip_code="ABCDE",
    number=123,
)

In [27]:
cat = CatRequest(
    name="Lévy",
    age=3,
    address=address
)

In [28]:

process(address)  # Just got an address from Wonderland


Just got an address from Wonderland


In [29]:
process(cat)  # Processing Lévy the cat!

Processing Lévy the cat!


In [30]:
# NotImplementedError: I don't know how to process <class 'str'>
process("something else")

NotImplementedError: I don't know how to process <class 'str'>

In [33]:
class Blog(BaseModel):
    title: str 
    description: Optional[str]=None
    is_active: bool

In [37]:
Blog(title="My First Blog", is_active=True)
# Output : Blog(title='My First Blog', description=None, is_active=True)


Blog(title='My First Blog', description=None, is_active=True)